# 🧠 RAG 기반 전세사기피해 법률 상담 QA 파이프라인

이 노트북은 **법률 문서(`law_1.docx`, `law_2.docx`)를 기반으로 한 Retrieval-Augmented Generation (RAG)** 파이프라인을 구현합니다.  
프로세스는 아래와 같습니다:

1. 문서 로드 및 전처리
2. 임베딩 및 벡터스토어 생성
3. RetrievalQA 구현

> **[목표]**  
> 법률 문서를 자동으로 검색·요약하여 전세사기 관련 법률 조언을 제공하는  
> RAG 기반 챗봇 파이프라인 구축



In [1]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone


## 환경변수 읽어오기
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

## 문서 읽고 분할 ##################################
## 문서 파일 목록
# doc_paths = ['law_1.docx', 'law_2.docx']

# ## 문서 로드(읽어오기)
# documents = []

# for path in doc_paths:
#     loader = Docx2txtLoader(path)
#     documents.extend(loader.load())

# ## 청크 분할 설정
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1500,
#     chunk_overlap=200,
# )

# ## 문서 분할
# document_list = text_splitter.split_documents(documents)


## 임베딩 -> 벡터 스토어(데이터베이스)에 저장
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'law'

## 파인콘: 저장
# database =  PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name,
# )

## 저장된 인덱스 가져오기
## [방법 1]
# database = PineconeVectorStore(
#     index=pc.Index(index_name),
#     embedding=embedding,
# )

## [방법 2]
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)

## RetrievalQA
llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

qa_chain = (
    {
        'context': database.as_retriever() | format_docs,
        'question': RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# qa_chain.invoke('전세사기피해자 대상을 알려주세요.')
qa_chain.invoke('전세사기피해자 임대인을 알려주세요.')

c:\Users\hi\AppData\Local\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'전세사기피해자 임대인은 특별법에서 "임대인"으로 정의된 인물이나 기관을 말하며, 실제로 주택의 임대에 관련된 업무를 처리하거나 임대인을 대리하는 자, 임대인을 위한 중개업자 등이 포함됩니다. 또한, 다수 임대인의 배후에 있는 동일인이나 이를 지배하는 조직도 포함됩니다. 추가로, 이를 기반으로 임대차 계약을 맺은 자나 그 조직도 전세사기피해자법에서 정의하는 임대인으로 간주됩니다.'

In [2]:
%whos

Variable                         Type                   Data/Info
-----------------------------------------------------------------
ChatOpenAI                       ModelMetaclass         <class 'langchain_openai.<...>_models.base.ChatOpenAI'>
Docx2txtLoader                   ABCMeta                <class 'langchain_communi<...>document.Docx2txtLoader'>
LANGCHAIN_API_KEY                str                    lsv2_pt_4c2ea048f4b64e489<...>100e74f13518f9_2579b3041d
OpenAIEmbeddings                 ModelMetaclass         <class 'langchain_openai.<...>s.base.OpenAIEmbeddings'>
PINECONE_API_KEY                 str                    pcsk_2v8RCj_2EuL5eWhJori2<...>a2uBh6NhmcE7rivqJVefzeZR3
Pinecone                         ABCMeta                <class 'pinecone.control.pinecone.Pinecone'>
PineconeVectorStore              ABCMeta                <class 'langchain_pinecon<...>res.PineconeVectorStore'>
RecursiveCharacterTextSplitter   ABCMeta                <class 'langchain_text_sp<...>veCharact

In [ ]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


## 환경변수 읽어오기 ############################################
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

## 벡터 스토어(데이터베이스)에서 인덱스 가져오기 ###############
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'law'

## 저장된 인덱스 가져오기
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)

## RetrievalQA ##################################################
llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

qa_chain = (
    {
        'context': database.as_retriever() | format_docs,
        'question': RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# qa_chain.invoke('전세사기피해자 대상을 알려주세요.')
qa_chain.invoke('전세사기피해자 임대인을 알려주세요.')